O algoritmo K-Nearest Neighbors (KNN) é um modelo de aprendizado supervisionado baseado na similaridade entre os dados. Ele funciona classificando um novo ponto de acordo com os k vizinhos mais próximos no conjunto de treinamento.

Como o KNN Funciona

- Escolha do valor de K: Determina quantos vizinhos mais próximos serão considerados para classificar um novo dado.
- Cálculo da distância: Para cada novo ponto, calculamos sua distância para todos os pontos do conjunto de treinamento. No nosso caso, utilizamos a distância Euclidiana.
- Identificação dos K vizinhos mais próximos: Selecionamos os K pontos mais próximos ao novo ponto de teste.
- Votação majoritária: O novo ponto recebe a classificação mais comum entre seus vizinhos.

Funções Implementadas

- distancia(v, w): Calcula a distância Euclidiana entre dois pontos.
- voto_majoritario(rotulos): Determina a classe do novo ponto com base nos vizinhos mais próximos.
- knn_classify(k, pontos_rotulados, novo_ponto): Implementa o KNN, organizando os pontos por proximidade, selecionando os k mais próximos e aplicando a votação majoritária.

Aplicação no Conjunto de Dados de Frutas

Para avaliar o modelo, criamos um conjunto de dados sintético representando frutas, onde cada fruta possui dois atributos:
- Peso (g): Varia entre 50g e 300g.
- Doçura (escala de 1 a 10).

As frutas foram classificadas com base nesses atributos:
- Banana: Se o peso for maior que 150g e a doçura maior que 5.
- Maçã: Caso contrário.

In [40]:
import random
from knn import LabeledPoint

# Função para gerar dados 
def gerar_dados_frutas(num_pontos=800):
    dados = []
    for _ in range(num_pontos):
        # Gerando características: peso (em gramas) e doçura (escala de 1 a 10)
        peso = random.uniform(50, 300)  # peso entre 50g e 300g
        doce = random.uniform(1, 10)  # doçura entre 1 e 10
        
        # Rótulo baseado em regras simples: Maçã ou Banana
        if peso > 150 and doce > 5:
            label = 'Banana'
        else:
            label = 'Maçã'
        
        dados.append(LabeledPoint([peso, doce], label))
    return dados

# Gerando dados para testar no modelo
dados_frutas = gerar_dados_frutas()

# Mostrando os primeiros 5 dados gerados
dados_frutas[:5]

[LabeledPoint(point=[87.30043571026253, 1.835123759598527], label='Maçã'),
 LabeledPoint(point=[129.60122232635698, 4.172818614239817], label='Maçã'),
 LabeledPoint(point=[151.0366667842142, 7.311646218291517], label='Banana'),
 LabeledPoint(point=[191.55333419648827, 1.8243734186864518], label='Maçã'),
 LabeledPoint(point=[156.08294710400844, 6.025351769592593], label='Banana')]

In [ ]:
from typing import TypeVar, Tuple, List

X = TypeVar('X')  # tipo genérico para representar um ponto de dados

def dividir_dados(dados: List[X], prop: float) -> Tuple[List[X], List[X]]:
    """Divide os dados em frações [prop, 1 - prop]"""
    dados = dados[:]                    # Faz uma cópia rasa
    random.shuffle(dados)               # porque shuffle modifica a lista.
    corte = int(len(dados) * prop)  # Usa a prop para encontrar o ponto de corte
    return dados[:corte], dados[corte:]

fruit_train, fruit_test = dividir_dados(dados_frutas, 0.7)
assert len(fruit_train) == 0.7 * 800
assert len(fruit_test) == 0.3 * 800
len(fruit_train), len(fruit_test)

(560, 240)

## Testando o Modelo

In [ ]:
from collections import defaultdict
from knn import knn_classify

# Inicializando a matriz de confusão e o contador de acertos
confusion_matrix = defaultdict(int)
acertos = 0

# Aplicando o modelo
for fruta in fruit_test:
    previsto = knn_classify(5, fruit_train, fruta.point)
    real = fruta.label

    if previsto == real:
        acertos += 1

    confusion_matrix[(previsto, real)] += 1

# Calculando a precisão e mostrando a matriz de confusão
accuracy = acertos / len(fruit_test)
print(accuracy, confusion_matrix, sep="\n")


0.9583333333333334
defaultdict(<class 'int'>, {('Maçã', 'Maçã'): 155, ('Banana', 'Banana'): 75, ('Banana', 'Maçã'): 6, ('Maçã', 'Banana'): 4})


## Com scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Separar os dados em características (X) e rótulos (y)
X = [fruta.point for fruta in dados_frutas]
y = [fruta.label for fruta in dados_frutas]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Criar o modelo KNN com scikit-learn
knn_sklearn = KNeighborsClassifier(n_neighbors=5)

# Treinar o modelo
knn_sklearn.fit(X_train, y_train)

# Fazer previsões
previsoes = knn_sklearn.predict(X_test)

# Calculando a precisão e mostrando a matriz de confusão
print(accuracy_score(y_test, previsoes), confusion_matrix(y_test, previsoes), sep="\n")

0.9708333333333333
[[ 70   4]
 [  3 163]]


Depois de comparar nosso modelo criado do zero com a versão do scikit-learn, obtivemos aproximadamente os mesmos resultados!

Isso mostra que os algoritmos de machine learning não são tão difíceis de entender quanto parecem.

Quando importamos um modelo pronto de uma biblioteca, nem sempre sabemos o que acontece por trás dos panos. Você pode ter pensado que o KNN envolvia muitos códigos e matemática complexa, mas, como vimos aqui, não é nenhum bicho de sete cabeças! No fim das contas, é um modelo bem simples e não precisamos de muitas linhas de código para implementá-lo. Claro, a versão da biblioteca é mais otimizada e oferece diversos parâmetros para ajustes finos, mas entender a implementação do zero nos dá mais controle e conhecimento sobre seu funcionamento.